
# Table of Contents <a name="TOC"></a> 

1.  [HW  Problems](#1)   
    1.0.  [HW3.1](#1.1)   
    1.2.  [HW3.2](#1.2)   
    1.3.  [HW3.3](#1.3)    
    1.4.  [HW3.4](#1.4)    
    

In [1]:
!pwd

/home/cloudera/anaconda2


In [4]:
! find / -name "hadoop*streaming*" -print 1> tmp.hadoopJar 2> /dev/null

In [5]:
!cat tmp.hadoopJar

/usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-mr1.jar
/usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.8.0.jar
/usr/lib/oozie/oozie-sharelib-yarn/lib/mapreduce-streaming/hadoop-streaming.jar
/usr/lib/oozie/oozie-sharelib-mr1/lib/mapreduce-streaming/hadoop-streaming.jar
/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.8.0.jar
/usr/lib/hadoop-mapreduce/hadoop-streaming.jar
/usr/share/doc/hadoop-2.6.0+cdh5.8.0+1589/hadoop-streaming


In [6]:
!ls -l /usr/lib/hadoop-mapreduce/hadoop-streaming.jar

lrwxrwxrwx 1 root root 35 Aug 10 13:00 /usr/lib/hadoop-mapreduce/hadoop-streaming.jar -> hadoop-streaming-2.6.0-cdh5.8.0.jar


In [2]:
!mkdir HW3

In [7]:
%cd HW3

/home/cloudera/anaconda2/HW3


In [8]:
!curl 'http://www.gutenberg.org/cache/epub/11/pg11.txt' -o alicesTExtFilename.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  163k  100  163k    0     0  38323      0  0:00:04  0:00:04 --:--:-- 38412


In [9]:
!head alicesTExtFilename.txt

# 3.  HW3  <a name="3"></a>

[Back to Table of Contents](#TOC)

* Dont forget to create a new subdirectory for this homework (on your local machine/virtual machine, and on HDFS)
---
## 3.  HW3.1  <a name="1.1"></a>
[Back to Table of Contents](#TOC)

In Hadoop streaming write code to determing the top-10 most frequent  using  words present in the text version of Alice’s Adventures in Wonderland. (You can obtain a free plain text version of the book, along with many others, from [here](http://www.gutenberg.org.). First use one reducer and report your result. Then develop a solution when you use 3 reducers? 


* Please leverage the following notebook for this HW problem: [Total Sort Notebook](http://nbviewer.jupyter.org/urls/dl.dropbox.com/s/44idk0xz2wyu45l/total-sort-guide.ipynb)

For your result please  create a text file named __alice_sorted_words.txt__ containing an  listing of all the words, and the number of times each occurs, in the text version of Alice’s Adventures in Wonderland in the following sort order: WordCount descreasing, and where words that have the same count, they be ordered in alphabetical increasing. (You can obtain a free plain text version of the book, along with many others, from here The first 10 lines of your output file should look something like this:

In [74]:
%%writefile mapper.py
#!/usr/bin/env python

import sys
#sys.stderr.write("reporter:counter:Tokens,Total,1") # NOTE missing the carriage return so wont work
sys.stderr.write("reporter:counter:Mapper Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing my message...how are  you\n")

import re
word_list=[]
for line in sys.stdin:
    clean_line = re.sub('\W+', " ",line )
    clean_line = clean_line.lower()
    words = clean_line.split() 
    for word in words:
        print '%s\t%s' % (word, 1)

Overwriting mapper.py


In [73]:
%%writefile reducer.py
#!/usr/bin/env python

import sys

cur_key = None
cur_count = 0
sys.stderr.write("reporter:counter:Reducer Counters,Calls,1\n")
for line in sys.stdin:
    key, value = line.split()
    if key == cur_key:
        cur_count += int(value)
    else:
        if cur_key:
            print '%s\t%s' % (cur_key, cur_count)
        cur_key = key
        cur_count = int(value)

print '%s\t%s' % (cur_key, cur_count)

Overwriting reducer.py


In [69]:
!hdfs dfs -ls

Found 12 items
drwxr-xr-x   - cloudera cloudera          0 2016-08-30 22:28 HW2
drwxr-xr-x   - cloudera cloudera          0 2016-08-31 09:28 HWTwoHDFS
-rw-r--r--   1 cloudera cloudera        292 2016-08-31 19:22 London.txt
drwxr-xr-x   - cloudera cloudera          0 2016-08-31 19:02 UnitOut
-rw-r--r--   1 cloudera cloudera         40 2016-08-31 18:54 UnitTest.txt
drwxr-xr-x   - cloudera cloudera          0 2016-08-30 22:21 WordCount
drwxr-xr-x   - cloudera cloudera          0 2016-09-02 03:25 alcountnew
drwxr-xr-x   - cloudera cloudera          0 2016-08-31 19:04 alicecaseout
drwxr-xr-x   - cloudera cloudera          0 2016-09-02 03:16 alicecounts
drwxr-xr-x   - cloudera cloudera          0 2016-08-31 11:42 aliceout
-rw-r--r--   1 cloudera cloudera     167518 2016-08-31 09:55 alicesTExtFilename.txt
drwxr-xr-x   - cloudera cloudera          0 2016-08-31 19:31 londonout


In [70]:
!chmod a+x mapper.py
!chmod a+x reducer.py

In [75]:
!echo "foo foo quux labs foo bar quux" | ./mapper.py 

reporter:counter:Mapper Counters,Calls,1
reporter:status:processing my message...how are  you
foo	1
foo	1
quux	1
labs	1
foo	1
bar	1
quux	1


In [76]:
!echo "foo foo quux labs foo bar quux" | ./mapper.py | sort -k1,1 | ./reducer.py| sort -k2,2nr 

reporter:counter:Reducer Counters,Calls,1
reporter:counter:Mapper Counters,Calls,1
reporter:status:processing my message...how are  you
foo	3
quux	2
bar	1
labs	1


In [78]:
!hdfs dfs -rm -r alicecounts

Deleted alicecounts


In [79]:
!hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -file "/home/cloudera/anaconda2/HW3/mapper.py"\
    -mapper /home/cloudera/anaconda2/HW3/mapper.py\
    -file "/home/cloudera/anaconda2/HW3/reducer.py"\
    -reducer /home/cloudera/anaconda2/HW3/reducer.py\
    -input alicesTExtFilename.txt\
    -output alicecounts\
    -numReduceTasks 1

16/09/03 03:44:49 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/home/cloudera/anaconda2/HW3/mapper.py, /home/cloudera/anaconda2/HW3/reducer.py] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.8.0.jar] /tmp/streamjob8947496028250894535.jar tmpDir=null
16/09/03 03:44:50 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/09/03 03:44:51 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/09/03 03:44:52 INFO mapred.FileInputFormat: Total input paths to process : 1
16/09/03 03:44:52 INFO mapreduce.JobSubmitter: number of splits:2
16/09/03 03:44:52 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1472659817953_0019
16/09/03 03:44:53 INFO impl.YarnClientImpl: Submitted application application_1472659817953_0019
16/09/03 03:44:53 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1472659817953_0019/
16/09/03 03:44:53 INFO mapred

In [80]:
!hdfs dfs -cat alicecounts/part-0000* | sort -k2,2nr -k1,1

the	1818
and	940
to	809
a	690
of	631
it	610
she	553
i	543
you	481
said	462
in	431
alice	403
was	358
that	330
as	274
her	248
with	228
at	227
s	219
t	218
on	204
all	200
this	181
for	179
had	178
but	175
be	167
not	166
or	155
they	155
so	152
very	145
what	142
is	135
he	128
little	128
out	118
if	116
one	106
up	103
about	102
down	102
there	101
no	100
do	98
his	96
then	94
gutenberg	93
know	88
them	88
project	87
have	85
like	85
were	85
again	83
herself	83
went	83
would	83
when	80
by	78
could	78
any	76
queen	75
thought	74
are	73
can	73
off	73
how	72
time	71
your	71
see	70
me	68
into	67
who	66
did	63
its	63
king	63
m	63
well	63
an	61
don	61
now	60
turtle	59
began	58
my	58
way	58
ll	57
tm	57
hatter	56
mock	56
which	56
gryphon	55
quite	55
must	54
other	54
think	53
work	53
just	52
much	52
only	52
some	52
their	52
first	51
from	51
here	51
rabbit	51
say	51
go	50
head	50
more	50
thing	49
never	48
voice	48
come	47
such	47
1	46
get	46
got	45
looked	45
oh	45
mouse	44
ve	44
after	43
him	43
we	43
duchess	4

In [46]:
!hdfs dfs -cat alicecounts/part-0000* 

0	1
000	1
1	46
11	4
1500	1
1887	1
1994	1
2	3
20	2
2001	1
2008	1
2011	1
25	1
3	11
30	1
4	5
4557	1
5	4
50	1
501	2
596	1
6	2
60	1
6221541	1
64	1
7	3
8	4
801	1
809	1
84116	1
9	3
90	2
99712	1
_i_	2
a	690
abide	2
able	1
about	102
above	3
absence	1
absurd	2
accept	1
acceptance	1
accepted	2
accepting	1
access	10
accessed	1
accessible	1
accident	2
accidentally	1
accordance	2
account	1
accounting	1
accounts	1
accusation	1
accustomed	1
ache	1
across	5
act	1
active	2
actual	1
actually	1
ada	1
added	23
adding	1
addition	1
additional	4
additions	1
address	1
addressed	2
addresses	1
addressing	1
adjourn	1
adoption	1
advance	3
advantage	3
adventures	12
advice	2
advisable	2
advise	1
affair	1
affectionately	1
afford	1
afore	1
afraid	12
after	43
afterwards	2
again	83
against	10
age	4
aged	1
agent	1
ago	2
agony	1
agree	11
agreed	1
agreement	18
ah	5
ahem	1
air	15
airs	1
ak	1
alarm	2
alarmed	1
alas	4
alice	403
alive	3
all	200
allow	4
almost	8
alone	5
along	6
aloud	5
already	3
also	4
alteration	1
altered	1
al

In [49]:
!hdfs dfs -ls alicecounts


Found 11 items
drwxr-xr-x   - cloudera cloudera          0 2016-08-30 22:28 HW2
drwxr-xr-x   - cloudera cloudera          0 2016-08-31 09:28 HWTwoHDFS
-rw-r--r--   1 cloudera cloudera        292 2016-08-31 19:22 London.txt
drwxr-xr-x   - cloudera cloudera          0 2016-08-31 19:02 UnitOut
-rw-r--r--   1 cloudera cloudera         40 2016-08-31 18:54 UnitTest.txt
drwxr-xr-x   - cloudera cloudera          0 2016-08-30 22:21 WordCount
drwxr-xr-x   - cloudera cloudera          0 2016-08-31 19:04 alicecaseout
drwxr-xr-x   - cloudera cloudera          0 2016-09-02 03:16 alicecounts
drwxr-xr-x   - cloudera cloudera          0 2016-08-31 11:42 aliceout
-rw-r--r--   1 cloudera cloudera     167518 2016-08-31 09:55 alicesTExtFilename.txt
drwxr-xr-x   - cloudera cloudera          0 2016-08-31 19:31 londonout


In [52]:
!hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D stream.num.map.output.key.fields=2 \
-D stream.map.output.field.separator="\t" \
-D mapreduce.partition.keycomparator.options="-k2,2nr -k1,1" \
-mapper /bin/cat \
-reducer /bin/cat \
-input alicecounts -output alcountnew \

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.8.0.jar] /tmp/streamjob6591051631854620552.jar tmpDir=null
16/09/02 03:25:02 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/09/02 03:25:03 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/09/02 03:25:03 INFO mapred.FileInputFormat: Total input paths to process : 1
16/09/02 03:25:03 INFO mapreduce.JobSubmitter: number of splits:2
16/09/02 03:25:03 INFO Configuration.deprecation: mapred.output.key.comparator.class is deprecated. Instead, use mapreduce.job.output.key.comparator.class
16/09/02 03:25:04 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1472659817953_0018
16/09/02 03:25:04 INFO impl.YarnClientImpl: Submitted application application_1472659817953_0018
16/09/02 03:25:04 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1472659817953_0018/
16/09/02 03:25:04 INFO mapreduce.Job: Running job: job_147265981795

In [53]:
!hdfs dfs -ls alcountnew

Found 2 items
-rw-r--r--   1 cloudera cloudera          0 2016-09-02 03:25 alcountnew/_SUCCESS
-rw-r--r--   1 cloudera cloudera      31730 2016-09-02 03:25 alcountnew/part-00000


In [55]:
!hdfs dfs -cat alcountnew/part-0*

the	1818	
and	940	
to	809	
a	690	
of	631	
it	610	
she	553	
i	543	
you	481	
said	462	
in	431	
alice	403	
was	358	
that	330	
as	274	
her	248	
with	228	
at	227	
s	219	
t	218	
on	204	
all	200	
this	181	
for	179	
had	178	
but	175	
be	167	
not	166	
or	155	
they	155	
so	152	
very	145	
what	142	
is	135	
he	128	
little	128	
out	118	
if	116	
one	106	
up	103	
about	102	
down	102	
there	101	
no	100	
do	98	
his	96	
then	94	
gutenberg	93	
know	88	
them	88	
project	87	
have	85	
like	85	
were	85	
again	83	
herself	83	
went	83	
would	83	
when	80	
by	78	
could	78	
any	76	
queen	75	
thought	74	
are	73	
can	73	
off	73	
how	72	
time	71	
your	71	
see	70	
me	68	
into	67	
who	66	
did	63	
its	63	
king	63	
m	63	
well	63	
an	61	
don	61	
now	60	
turtle	59	
began	58	
my	58	
way	58	
ll	57	
tm	57	
hatter	56	
mock	56	
which	56	
gryphon	55	
quite	55	
must	54	
other	54	
think	53	
work	53	
just	52	
much	52	
only	52	
some	52	
their	52	
first	51	
from	51	
here	51	
rabbit	51	
say	51	
go	50	
head	50	
more	50	
thing	49	
neve